In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from glob import glob
from scipy.stats import norm
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.palettes import Bright6, Category20
from bokeh.transform import factor_cmap

output_notebook()

Loading BokehJS ...

Data from IA download is stored in tsv files that list which Item was downloaded when, how many seconds that took, and whether an error occurred. Different log files equal different runs, either with slightly different configurations. E.g. checking whether the file on disk has the correct md5 checksum (slow) or not. Number of paralle downloads. Whether we're downloading WIDE15 or WIDE6.

In [68]:
def read_table(files):
    dfs = []
    for filename in files:
        df = pd.read_table(filename,
                           names=['ts', 'item', 'file', 'path', 'size', 'download_sec', 'md5', 'error'],
                           parse_dates=[0])
        df['logfile'] = filename
        dfs.append(df)
    return pd.concat(dfs)

data = read_table(
    file_name
    for file_name in glob('ia-download.log.*')
    if datetime.strptime(file_name.split('.')[-1], '%Y%m%d%H%M%S') > datetime(2023,2,1)
)

In [69]:
data

,ts,item,file,path,size,download_sec,md5,error,logfile
0,2023-02-23 11:37:11.695005,WIDE-20150315001948-crawl800,WIDE-20150315005646-00028.warc.gz,./WIDE-20150315001948-crawl800/WIDE-2015031500...,1.000004e+09,84.0,cabb12e072772465a8131e279d1da753,NaN,ia-download.log.20230223113546
1,2023-02-23 11:37:12.416287,WIDE-20150315043804-crawl422,WIDE-20150315044302-00086.warc.gz,./WIDE-20150315043804-crawl422/WIDE-2015031504...,1.000035e+09,84.0,aa01ec2fd6ebdc0b043223aeb8f002d3,NaN,ia-download.log.20230223113546
2,2023-02-23 11:37:12.687914,WIDE-20150315023406-crawl803,WIDE-20150315025610-00051.warc.gz,./WIDE-20150315023406-crawl803/WIDE-2015031502...,1.000004e+09,84.0,c04231de44879a705d5f51f2d130eece,NaN,ia-download.log.20230223113546
3,2023-02-23 11:37:24.896855,WIDE-20150315043804-crawl422,WIDE-20150315050515-00090.warc.gz,./WIDE-20150315043804-crawl422/WIDE-2015031505...,1.001301e+09,97.0,6d6754be83dd218d8bbaf601f8a6a4df,NaN,ia-download.log.20230223113546
4,2023-02-23 11:37:40.319762,WIDE-20150315031230-crawl338,WIDE-20150315040351-00045.warc.gz,./WIDE-20150315031230-crawl338/WIDE-2015031504...,1.000035e+09,112.0,5eaff5163964c2131d09fe2634e6256f,NaN,ia-download.log.20230223113546
...,...,...,...,...,...,...,...,...,...
164064,2023-02-27 21:18:00.802069,WIDE-20150412061311-crawl809,WIDE-20150412062210-02026.warc.gz,./WIDE-20150412061311-crawl809/WIDE-2015041206...,1.051029e+09,1244.0,ff55feea996550cd3b87c02e223c36d7,NaN,ia-download.log.20230223113546
164065,2023-02-27 21:18:08.816644,WIDE-20150412051041-crawl424,WIDE-20150412055318-05816.warc.gz,./WIDE-20150412051041-crawl424/WIDE-2015041205...,1.000002e+09,2424.0,98df646805878b195603ac992eaa862e,NaN,ia-download.log.20230223113546
164066,2023-02-27 21:18:13.056730,WIDE-20150412051041-crawl424,WIDE-20150412060114-05818.warc.gz,./WIDE-20150412051041-crawl424/WIDE-2015041206...,1.000003e+09,2414.0,b3f2537e6563786f3c2366eebd722931,NaN,ia-download.log.20230223113546
164067,2023-02-27 21:18:13.145554,WIDE-20150412061427-crawl424,WIDE-20150412061427-05822.warc.gz,./WIDE-20150412061427-crawl424/WIDE-2015041206...,1.000003e+09,1244.0,068bc413a2dc65d5ff5227d316d8cda7,NaN,ia-download.log.20230223113546


In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164069 entries, 0 to 164068
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   ts            164069 non-null  datetime64[ns]
 1   item          164069 non-null  object        
 2   file          164069 non-null  object        
 3   path          149912 non-null  object        
 4   size          149912 non-null  float64       
 5   download_sec  149912 non-null  float64       
 6   md5           149912 non-null  object        
 7   error         14157 non-null   object        
 8   logfile       164069 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 11.3+ MB


In [71]:
data_per_hour = data.groupby(['logfile', data['ts'].dt.floor('H')])

# Download time

In [72]:
p = figure(title="Download time (binned)",
          x_axis_label="Seconds",
          y_axis_label="Density")
# downloads = data[data['error'].isna() & (data['logfile'] == 'ia-download.log.20221231114150')]
downloads = data
bins = np.linspace(0, 3600, 200)
hist, edges = np.histogram(downloads['download_sec'], density=True, bins=bins)
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:])
show(p)


Seconds it takes to download a file. Looks like there are multiple humps hidden in this graph. I expect that if you'd graph them per log file you'll see slightly different means.

In [73]:
# Scatter plot of all individual downloads. Too large to load now.
# from bokeh.models import DatetimeTickFormatter
# p = figure(title="Download time (over time)",
#            x_axis_type="datetime",
#            x_axis_label="Time",
#            y_axis_label="Duration (seconds)",
#            y_range=(0, 2*3600),
#            sizing_mode="stretch_width",
#            height=250,)
# p.circle(x='ts', y='download_sec', fill_alpha=0.1, color=factor_cmap('logfile', palette=Bright6, factors=data['logfile'].unique()), source=ColumnDataSource(data))
# p.line(x='ts', y='avg_download_sec',
#        source=ColumnDataSource(data_per_hour['download_sec'].mean().reset_index(name='avg_download_sec')),
#        color='red', width=2)
# show(p)

In [74]:
Colors = {
    **Category20,
    1: ('#1f77b4',),
    2: ('#1f77b4', '#aec7e8'),
}

In [75]:
def lines(p, x, y, color_field, data, *, palette=Colors, legend=False):
    """Helper function to plot groups as separete lines"""
    groups = data[color_field].unique()
    for color, group in zip(palette[len(groups)], groups):
        p.line(x, y, color=color, source=ColumnDataSource(data[data[color_field]==group]), **(dict(legend_label=group) if legend else dict()))

In [76]:
errors_per_hour = data_per_hour['error'].count().reset_index(name='errors')

requests_per_hour = data_per_hour.size().reset_index(name='requests')

errors_per_hour['error_rate'] = errors_per_hour['errors'] / requests_per_hour['requests']

p = figure(title="Fraction of downloads that fail (over time, per hour)",
           x_axis_type="datetime",
           x_axis_label="Hour",
           y_axis_label="Error rate",
           sizing_mode="stretch_width",
           height=250,)
lines(p, 'ts', 'error_rate', 'logfile', errors_per_hour)
show(p)

Error rate seems to be about 10% of all downloads when downloading WIDE15 with 128 or 256 threads. The yellow bit at the end is WIDE6 with 64 threads, eventually failing on a certificate issue. Graph below splits errors out per category.

In [77]:
def classify_error(error):
    if error == 'None: None' or 'Max retries exceeded' in error:
        return 'timeout'
    elif error == 'md5 mismatch':
        return 'md5 mismatch'
    elif error.startswith('[Errno None: None] '):
        return 'other'
    else:
        return error

data['error_norm'] = data['error'].map(classify_error, na_action='ignore')

errors_per_hour = data.groupby(['error_norm', data['ts'].dt.floor('H')])['ts'].count().reset_index(name='error_count')
p = figure(title="Errors per category (per hour)",
           x_axis_type="datetime",
           x_axis_label="Hour",
           y_axis_label="Error count",
           sizing_mode="stretch_width",
           height=250,)
lines(p, 'ts', 'error_count', 'error_norm', errors_per_hour, legend=True)
p.legend.location = 'top_left'
show(p)

Initially the most occurring problem were timeouts, but then as more files finished downloading correctly and failed downloads were re-attempted, the md5 errors became more prevalent. I should check whether the error rate is consistent per file or per item really. Error count is much lower at the end where I was doing md5 checking of existing files (so less actual downloading) and downloading WIDE6 (with 64 in parallel as opposed to 256, so again less actual downloading.)

In [78]:
errors_per_hour.groupby('error_norm').sum(numeric_only=True)

,error_count
error_norm,
Bad Gateway,11
md5 mismatch,13731
other,17
timeout,398


In [79]:
bytes_per_hour = data_per_hour['size'].sum().reset_index(name='amount') # size is NA on error, files that fail md5 check are not counted
p = figure(title="Amount of data downloaded (over time, per hour)",
           x_axis_type="datetime",
           x_axis_label="Hour",
           y_axis_label="Downloaded data",
           sizing_mode="stretch_width",
           height=250,)
lines(p, 'ts', 'amount', 'logfile', bytes_per_hour)
show(p)

Going by eye, average seems to be about 1TB/hour with the 256 parallel downloads I was using around Chirstmas. New year started with checking existing files (so fewer downloads) and then completing WIDE6 (again fewer downloads as I was checking md5 of existing files and only using 64 downloads in parallel.)

In [80]:
f"{data[data['error'].isna()]['item'].unique().size} files downloaded"

'18054 files downloaded'

In [81]:
f"{data['ts'].dt.floor('H').unique().size / 24:.1f} days of downloading"

'4.5 days of downloading'

In [82]:
f"{data_per_hour['size'].sum().mean() / (1000 * 1000 * 1000):.0f}GB per hour on average"

'1516GB per hour on average'

Above average is a bad estimate for top download speed. It includes periods that I mostly did md5 checking of existing files, and only re-downloading files that did not pass this check.

In [83]:
p = figure(title="Number of files downloaded (over time, per hour)",
           x_axis_type="datetime",
           x_axis_label="Hour",
           y_axis_label="Downloaded files",
           sizing_mode="stretch_width",
           height=250,)
files_per_hour = data_per_hour['md5'].count().reset_index(name='file_count') # md5 is NA on errors
lines(p, 'ts', 'file_count', 'logfile', files_per_hour)
show(p)

Surprise surprise above graph closely matches the amount of data downloaded graph.